In [1]:
import mmrotate
import mmdet
import mmcv
from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector, show_result_pyplot
from mmrotate.models import build_detector
import numpy as np
import os
from PIL import Image
import time

# Create the 'models' folder if it doesn't exist
if not os.path.exists('models'):
    os.mkdir('models')

# Check if the files exist, and download them only if they don't
config_file = './models/oriented_rcnn_r50_fpn_1x_dota_le90.py'
checkpoint_file = './models/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth'

if not os.path.exists(config_file) or not os.path.exists(checkpoint_file):
    !mim download mmrotate --config oriented_rcnn_r50_fpn_1x_dota_le90 --dest ./models
else:
    print("Files already exist.")

# Set the device to be used for evaluation
device='cuda:0'

# Load the config
config = mmcv.Config.fromfile(config_file)
# Set pretrained to be None since we do not need pretrained model here
config.model.pretrained = None

# Initialize the detector
model = build_detector(config.model)

# Load checkpoint
checkpoint = load_checkpoint(model, checkpoint_file, map_location=device)

# Set the classes of models for inference
model.CLASSES = checkpoint['meta']['CLASSES']

# We need to set the model's cfg for inference
model.cfg = config

# Convert the model to GPU
model.to(device)
# Convert the model into evaluation mode
model.eval()

print("Model loaded: " + str(checkpoint_file))

/home/ubuntu/anaconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


/home/ubuntu/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
processing oriented_rcnn_r50_fpn_1x_dota_le90...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 MiB 2.4 MB/s eta 0:00:0000:0100:02
Successfully downloaded oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth to /home/ubuntu/FEX/demo/models
Successfully dumped oriented_rcnn_r50_fpn_1x_dota_le90.py to /home/ubuntu/FEX/demo/models


/home/ubuntu/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: ./models/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth
Model loaded: ./models/oriented_rcnn_r50_fpn_1x_dota_le90-6d2b2ce0.pth


In [2]:
def getObjects(modelname, model, result):
    objects = []

    if modelname == "orcnn":
        bbox_result, segm_result = result, None
        bboxes = np.vstack(bbox_result)

        bboxes = np.vstack(bbox_result) #boxes (ndarray): Bounding boxes (with scores), shaped (n, 5) or (n, 6).
        labels = [
            np.full(bbox.shape[0], i, dtype=np.int32)
            for i, bbox in enumerate(bbox_result)
        ]
        labels = np.concatenate(labels)

        
        for i, bbox in enumerate(bboxes):
            xc, yc, w, h, ag = bbox[:5]
            wx, wy = w / 2 * np.cos(ag), w / 2 * np.sin(ag)
            hx, hy = -h / 2 * np.sin(ag), h / 2 * np.cos(ag)
            #p1 = (xc - wx - hx, yc - wy - hy)
            #p2 = (xc + wx - hx, yc + wy - hy)
            #p3 = (xc + wx + hx, yc + wy + hy)
            #p4 = (xc - wx + hx, yc - wy + hy)
            x1 = xc - wx - hx
            y1 = yc - wy - hy
            x2 = xc + wx - hx
            y2 = yc + wy - hy
            x3 = xc + wx + hx
            y3 = yc + wy + hy
            x4 = xc - wx + hx
            y4 = yc - wy + hy
            
            obj = {
                "bounds": {
                    "x1": x1,
                    "y1": y1,
                    "x2": x2,
                    "y1": y2,
                    "x3": x3,
                    "y3": y3,
                    "x4": x4,
                    "y4": y4
                },
                "confidence": bbox[5],
                "tagName": model.CLASSES[labels[i]]
            }

            objects.append(obj)
    return objects

In [4]:
# Use the detector to do inference
import time
import numpy as np
from PIL import Image
start_time = time.time()

img = 'demo.jpg'
img_pil = Image.open("demo.jpg")
img_array = np.array(img_pil)
result = inference_detector(model, img_array)
end_time = time.time()

elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 0.6358942985534668 seconds


/home/ubuntu/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


In [5]:
np.shape(img_array)

(1024, 1024, 3)

In [11]:
import base64
import datetime
from datetime import datetime
with open('b64img.txt', 'r') as file:
    b64_image = file.read()

timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
frmt = "png"
name = f"orcnn_img_{timestamp}.{frmt}"
path = f"images/{name}"

with open(path, "wb") as fh:
    fh.write(base64.b64decode(b64_image))

im = Image.open(path)
np_array = np.asarray(im)

FileNotFoundError: [Errno 2] No such file or directory: 'images/orcnn_img_20240513122748285296.png'

In [8]:
# Convert the image to a NumPy array
np_array = np.asarray(im)

(968759,)